# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [31]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import matplotlib.lines as mlines

def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Load data

In [23]:
path = f"D:/YandexDisk/Work/bbd"

df_subject = pd.read_csv(f"{path}/ipop/subject.txt", index_col=0, delimiter="\t")
df_visit = pd.read_csv(f"{path}/ipop/visit.txt", index_col=0, delimiter="\t")
feats_visit = ['CollectionDate', 'Event', 'Event_Note1', 'Event_Note2', 'Event_Note3', 'SubStudy']
df_imm_sml = pd.read_csv(f"{path}/ipop/Targeted Assays/Targ.proteomics.csv", index_col=0)
df_imm_big = pd.read_csv(f"{path}/ipop/Targeted Assays/cytokine_abundance.txt", index_col=0, delimiter="\t")

feats_aliases = pd.read_excel(f"{path}/ipop/Targeted Assays/feats_aliases.xlsx", index_col=0)
feats_aliases.dropna(axis=0, how='any', inplace=True)
feats_aliases_dict = dict(zip(feats_aliases['ipop'], feats_aliases['gene']))
df_imm_big.rename(columns=feats_aliases_dict, inplace=True)
df_imm_sml.rename(columns=feats_aliases_dict, inplace=True)

feats_imm_slctd = pd.read_excel(f"{path}/immunology//feats_selected.xlsx", index_col=0).index.values
feats_cmn = list(set.intersection(set(feats_aliases_dict.values()), set(feats_imm_slctd)))

df_imm_big = df_imm_big.loc[:, feats_cmn]
df_imm_sml = df_imm_sml.loc[:, feats_cmn]

df_imm_big['Subject_ID'] = df_imm_big.index.str.split('-').str[0]
df_imm_sml['Subject_ID'] = df_imm_sml.index.str.split('-').str[0]

pheno_samples_sml = list(set.intersection(set(df_imm_sml['Subject_ID'].unique()), set(df_subject.index)))
for s in pheno_samples_sml:
    df_imm_sml.loc[df_imm_sml['Subject_ID'] == s, df_subject.columns.values] = df_subject.loc[s, df_subject.columns.values].values
pheno_ids_sml = df_imm_sml.index.intersection(df_visit.index).values
df_imm_sml.loc[pheno_ids_sml, feats_visit] = df_visit.loc[pheno_ids_sml, feats_visit]

pheno_samples_big = list(set.intersection(set(df_imm_big['Subject_ID'].unique()), set(df_subject.index)))
for s in pheno_samples_big:
    df_imm_big.loc[df_imm_big['Subject_ID'] == s, df_subject.columns.values] = df_subject.loc[s, df_subject.columns.values].values
pheno_ids_big = df_imm_big.index.intersection(df_visit.index).values
df_imm_big.loc[pheno_ids_big, feats_visit] = df_visit.loc[pheno_ids_big, feats_visit]

df_imm_our = pd.read_excel(f"{path}/immunology/002_central_vs_yakutia/data.xlsx", index_col=0)

# Big VS Our data

In [24]:
path_save = f"{path}/ipop/Targeted Assays/big_vs_our"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

df_imm_big = df_imm_big.loc[df_imm_big['Event'] == 'Healthy', :]
df_imm_big = df_imm_big[~df_imm_big.index.duplicated(keep='first')]
df_imm_our['Part'] = 'UNN'
df_imm_big['Part'] = 'IPOP'
df_imm = pd.concat([df_imm_our, df_imm_big])

In [ ]:
groups_ids = {
    'UNN': df_imm.index[(df_imm['Part'] == 'UNN')].values,
    'IPOP': df_imm.index[(df_imm['Part'] == 'IPOP')].values,
}

hist_bins = np.linspace(5, 115, 23)

n_rows = 2
n_cols = 1
fig_width = 5
fig_height = 6

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharex=True, sharey=False)

for group_id, group in enumerate(groups_ids.keys()):
    row_id, col_id = divmod(group_id, n_cols)
    
    histplot = sns.histplot(
        data=df_imm.loc[groups_ids[group], :],
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue='Sex',
        palette={'F': 'crimson', 'M': 'dodgerblue'},
        hue_order=['F', 'M'],
        ax=axs[row_id]
    )
    axs[row_id].set(xlim=(0, 120))
    axs[row_id].set_title(f"{group} ({len(groups_ids[group])})")
plt.savefig(f"{path_save}/hist_age.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/hist_age.pdf", bbox_inches='tight')
plt.close(fig)

## Select our data with same age distribution

In [26]:
ids_to = groups_ids['IPOP']
ids_from = groups_ids['UNN']

hist_bins = np.linspace(5, 115, 23)
age_counts, age_bin_edges = np.histogram(df_imm.loc[ids_to, 'Age'].values, bins=hist_bins)
age_prob = age_counts / len(df_imm.loc[ids_to, 'Age'].values)
bin_diff = age_bin_edges[1] - age_bin_edges[0]
df_imm.loc[ids_from, 'Prob Age (IPOP)'] = age_prob[np.rint((df_imm.loc[ids_from, 'Age'].values - age_bin_edges[0]) / (bin_diff + 0.0001)).astype(int)]

n_same_age = 200
index_from_to = df_imm.loc[ids_from, :].sample(n=n_same_age, replace=False, weights='Prob Age (IPOP)', random_state=1337).index
if index_from_to.is_unique:
    ids_from_to = index_from_to.values
else:
    print("Not unique index")

In [ ]:
df_imm.loc[ids_from_to, 'Part'] = 'UNN (same age)'

groups_ids = {
    'UNN (same age)': df_imm.index[(df_imm['Part'] == 'UNN (same age)')].values,
    'IPOP': df_imm.index[(df_imm['Part'] == 'IPOP')].values,
}

hist_bins = np.linspace(5, 115, 23)

n_rows = 2
n_cols = 1
fig_width = 5
fig_height = 6

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharex=True, sharey=False)

for group_id, group in enumerate(groups_ids.keys()):
    row_id, col_id = divmod(group_id, n_cols)
    
    histplot = sns.histplot(
        data=df_imm.loc[groups_ids[group], :],
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue='Sex',
        palette={'F': 'crimson', 'M': 'dodgerblue'},
        hue_order=['F', 'M'],
        ax=axs[row_id]
    )
    axs[row_id].set(xlim=(0, 120))
    axs[row_id].set_title(f"{group} ({len(groups_ids[group])})")
plt.savefig(f"{path_save}/hist_age_same.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/hist_age_same.pdf", bbox_inches='tight')
plt.close(fig)

### Calculate statistics

In [29]:
df_stat = pd.DataFrame(index=list(feats_cmn))
for feat in list(feats_cmn):
    vals = {}
    for group, ids in groups_ids.items():
        vals[group] = df_imm.loc[ids, feat].values
        df_stat.at[feat, f"mean_{group}"] = np.mean(vals[group])
        df_stat.at[feat, f"median_{group}"] = np.median(vals[group])
        df_stat.at[feat, f"q75_{group}"], df_stat.at[feat, f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
        df_stat.at[feat, f"iqr_{group}"] = df_stat.at[feat, f"q75_{group}"] - df_stat.at[feat, f"q25_{group}"]
    _, df_stat.at[feat, "mw_pval"] = mannwhitneyu(vals['UNN (same age)'], vals['IPOP'], alternative='two-sided')

_, df_stat.loc[feats_cmn, "mw_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[feats_cmn, "mw_pval"], 0.05, method='fdr_bh')
df_stat.sort_values([f"mw_pval_fdr_bh"], ascending=[True], inplace=True)
df_stat.to_excel(f"{path_save}/mw_same.xlsx", index_label='Features')

In [ ]:
df_stat['mw_pval_fdr_bh_log'] = -np.log10(df_stat['mw_pval_fdr_bh'])
df_stat['color'] = 'pink'
df_stat.loc[df_stat['mw_pval_fdr_bh'] < 0.05, 'color'] = 'red'
sns.set_theme(style='whitegrid')
fig, ax = plt.subplots(figsize=(4, 8))
barplot = sns.barplot(
    data=df_stat,
    y=df_stat.index.values,
    x='mw_pval_fdr_bh_log',
    edgecolor='black',
    palette=df_stat['color'].values,
    dodge=True,
    ax=ax
)
ax.set_xlabel(r"$-\log_{10}(\mathrm{p-value})$", fontsize=18)
ax.set_ylabel('', fontsize=20)
ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=16)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 16)
plt.savefig(f"{path_save}/mw_same.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/mw_same.pdf", bbox_inches='tight')
plt.close(fig)

In [ ]:
n_rows = 5
n_cols = 5
fig_width = 15
fig_height = 10

groups_colors = {
    'UNN (same age)': 'darkorange',
    'IPOP': 'forestgreen',
}

df_fig = df_imm.loc[list(set.union(set(groups_ids['UNN (same age)']), set(groups_ids['IPOP']))), :]

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={})
feats_sorted = df_stat.index.values
for f_id, f in enumerate(feats_sorted):
    row_id, col_id = divmod(f_id, n_cols)
    
    q01 = df_fig[f].quantile(0.01)
    q99 = df_fig[f].quantile(0.99)
    sns.violinplot(
        data=df_fig.loc[(df_fig[f] > q01) & (df_fig[f] < q99), :],
        x='Part',
        y=f,
        palette=groups_colors,
        scale='width',
        order=list(groups_colors.keys()),
        saturation=0.75,
        cut=0,
        linewidth=1.0,
        ax=axs[row_id, col_id],
        legend=False,
    )
    axs[row_id, col_id].set_ylabel(f)
    axs[row_id, col_id].set_xlabel('')
    axs[row_id, col_id].set(xticklabels=[]) 
    mw_pval = df_stat.at[f, "mw_pval_fdr_bh"]
    pval_formatted = [f'{mw_pval:.2e}']
    annotator = Annotator(
        axs[row_id, col_id],
        pairs=[('UNN (same age)', 'IPOP')],
        data=df_fig,
        x='Part',
        y=f,
        order=list(groups_colors.keys()),
    )
    annotator.set_custom_annotations(pval_formatted)
    annotator.configure(loc='outside')
    annotator.annotate()

legend_handles = [
    mlines.Line2D([], [], marker='o', linestyle='None', markeredgecolor='k', markerfacecolor=groups_colors['UNN (same age)'], markersize=10, label='UNN (same age)'),
    mlines.Line2D([], [], marker='o', linestyle='None', markeredgecolor='k', markerfacecolor=groups_colors['IPOP'], markersize=10, label='IPOP')
]
fig.legend(handles=legend_handles, bbox_to_anchor=(0.5, 1.0), loc="lower center", ncol=2, frameon=False, fontsize='large')
fig.tight_layout()    
plt.savefig(f"{path_save}/violins_same.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/violins_same.pdf", bbox_inches='tight')
plt.close(fig)
    